# Forecasting Snow Depth with Facebook Prophet


## Abstract
Facebook prophet is a great tool for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. Considering climate change, I was wondering is this tool can be used to predict snow evolution in the French Alps and particularly in Ski Resort.



## Introduction


## Data Analysis
In France, snow report data of 135 ski resorts are available freely from https://donneespubliques.meteofrance.fr/?fond=produit&id_produit=94&id_rubrique=32. These data are available as csv files and give several observation coming from snow sensor every 12 hours. The following map constructed with Folium https://python-visualization.github.io/folium map gives location of the different snow reports. Among them, we have some famous ski resort such as Tignes or l'Alpe d'huez.


In [1]:
import folium
m = folium.Map(location=[48.856578, 2.351828], zoom_start=6, tiles='Stamen Terrain')
folium.GeoJson(
    '../data/postesNivo.json',
    name='geojson'
).add_to(m)
m


Meteo France provides a file per month since 2010 with measurments coming from each of this stations. The documentation can be found here:

In [2]:
import pandas
measurments_201801 = pandas.read_csv('../data/nivo.201801.csv', delimiter=';', parse_dates=['date'])
# Remove rows without a snow height measurement
measurments_201801 = measurments_201801[measurments_201801.ht_neige != 'mq']
measurments_201801.head()


,numer_sta,date,haut_sta,dd,ff,t,td,u,ww,w1,...,ht_neige_alti,neige_fraiche,teneur_eau,grain_predom,grain_nombre,grain_diametr,homogeneite,m_vol_neige,Unnamed: 48,Unnamed: 49
0,7590,2018-01-01 07:10:00,1270.0,0,0.000000,274.250000,273.540000,95,0,7,...,mq,mq,mq,mq,mq,mq,mq,mq,NaN,NaN
1,7818,2018-01-01 07:10:00,1250.0,360,2.000000,271.150000,268.160000,80,0,7,...,mq,mq,mq,mq,mq,mq,mq,mq,NaN,NaN
2,7856,2018-01-01 06:30:00,1620.0,0,0.000000,270.350000,269.660000,95,70,0,...,3.100000,0.200000,mq,1,9,mq,0,80.000000,NaN,NaN
3,7888,2018-01-01 06:15:00,1800.0,0,0.000000,269.350000,267.950000,90,0,7,...,mq,mq,mq,1,2,mq,0,100.000000,NaN,NaN
4,7895,2018-01-01 06:40:00,1970.0,0,0.000000,268.150000,267.330000,94,70,7,...,mq,mq,mq,1,1,mq,mq,mq,NaN,NaN


The latitute, longitude, altitude, name of each station is also provided in a csv file and can be used to merge measurments.

In [3]:
stations = pandas.read_csv('../data/postesNivo.csv', delimiter=',')
stations.head()



,Latitude,Longitude,ID,Altitude,Nom
0,46.766000,6.359167,7392.0,1036,METABIEF_STATION
1,46.020833,6.970833,7393.0,2196,LE TOUR BALME
2,46.341167,6.708167,7454.0,1535,Bernex
3,45.247667,6.732667,7456.0,2166,Aussois
4,46.315333,6.673333,7457.0,790,VACHERESSE AUXI


We propose to aggregate all measurments in a dataframe and then to join it with the list of station. We thus finally get all observations of all stations in a single data frame.

In [4]:
import glob
path = '../data/'
nivo_files = [f for f in glob.glob(path + 'nivo*.csv')]
for f in nivo_files:
    print(f)



../data/nivo.201012.csv
../data/nivo.201210.csv
../data/nivo.201204.csv
../data/nivo.201402.csv
../data/nivo.201601.csv
../data/nivo.201403.csv
../data/nivo.201205.csv
../data/nivo.201211.csv
../data/nivo.201207.csv
../data/nivo.201401.csv
../data/nivo.201603.csv
../data/nivo.201602.csv
../data/nivo.201212.csv
../data/nivo.201206.csv
../data/nivo.201202.csv
../data/nivo.201410.csv
../data/nivo.201404.csv
../data/nivo.201612.csv
../data/nivo.201606.csv
../data/nivo.201809.csv
../data/nivo.201808.csv
../data/nivo.201607.csv
../data/nivo.201405.csv
../data/nivo.201411.csv
../data/nivo.201203.csv
../data/nivo.201201.csv
../data/nivo.201407.csv
../data/nivo.201605.csv
../data/nivo.201611.csv
../data/nivo.201610.csv
../data/nivo.201604.csv
../data/nivo.201412.csv
../data/nivo.201406.csv
../data/nivo.201703.csv
../data/nivo.201501.csv
../data/nivo.201307.csv
../data/nivo.201105.csv
../data/nivo.201111.csv
../data/nivo.201110.csv
../data/nivo.201104.csv
../data/nivo.201312.csv
../data/nivo.201

In [5]:
li = []
for filename in nivo_files:
    df = pandas.read_csv(filename, delimiter=';',parse_dates=['date'])
    li.append(df)
final_measurments = pandas.concat(li, ignore_index=True)



If we want to list all observations for Tignes we just only need to perform.

In [85]:
import pytemperature
tignes_measurements = final_measurments.loc[final_measurments.numer_sta==int(stations.loc[stations.Nom == 'AUTRANS-NIVO']['ID'])]
tignes_measurements


,numer_sta,date,haut_sta,dd,ff,t,td,u,ww,w1,...,ht_neige_alti,neige_fraiche,teneur_eau,grain_predom,grain_nombre,grain_diametr,homogeneite,m_vol_neige,Unnamed: 48,Unnamed: 49
7317,7488,2014-02-01 08:55:00,1260,0,0.000000,272.550000,270.500000,86,0,0,...,1.050000,0.000000,mq,2,4,mq,1,140.000000,NaN,NaN
7607,7488,2014-02-03 08:44:00,1260,180,1.000000,269.150000,268.190000,93,0,7,...,1.150000,0.030000,mq,2,2,mq,1,120.000000,NaN,NaN
7720,7488,2014-02-04 08:55:00,1260,310,1.000000,270.150000,269.030000,92,70,0,...,1.100000,0.010000,mq,7,6,mq,2,mq,NaN,NaN
7897,7488,2014-02-05 08:40:00,1260,180,2.000000,275.950000,271.590000,73,0,0,...,1.100000,0.000000,mq,6,3,mq,2,mq,NaN,NaN
8073,7488,2014-02-06 08:30:00,1260,0,0.000000,269.350000,268.670000,95,0,7,...,mq,mq,mq,2,2,mq,0,100.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117099,7488,2017-01-27 09:00:00,1260,0,0.000000,276.350000,269.770000,62,0,0,...,0.400000,0.000000,mq,7,3,mq,1,mq,NaN,NaN
117367,7488,2017-01-29 08:30:00,1260,0,0.000000,273.150000,271.710000,90,0,0,...,0.400000,0.000000,mq,7,3,mq,2,mq,NaN,NaN
117550,7488,2017-01-30 09:00:00,1260,0,0.000000,273.950000,270.890000,80,0,0,...,0.400000,0.000000,mq,7,3,mq,2,mq,NaN,NaN
117699,7488,2017-01-31 08:55:00,1260,0,0.000000,275.250000,274.240000,93,61,6,...,0.380000,0.000000,mq,6,6,mq,2,mq,NaN,NaN


## Forecasting Snow


In [82]:
from fbprophet import Prophet
from numpy import mean
from plotly.offline import plot
from plotly import graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

key = 't'

tignes_measurements = tignes_measurements[tignes_measurements[key] != 'mq']
tignes_measurements[key] = tignes_measurements[key].astype(float).apply(pytemperature.k2c)

daily_df = tignes_measurements[['date',key]].astype({
            'date': 'object',
            key: 'float64',
        })

daily_df = daily_df.set_index('date').resample('D').mean().interpolate()#fillna(0) #
daily_df.reset_index(drop=False,inplace=True)


df = pandas.DataFrame(columns=['ds','y'])
df['ds'] = daily_df['date']
df['y'] = daily_df[key]
df = df.set_index('ds')
df.reset_index(drop=False,inplace=True)
#daily_df = df.resample('D').apply(mean)
m1 = Prophet()#growth='logistic')
#df['cap']=0
df.columns = ["ds", "y"]
#df = df.reset_index()
m1.fit(df)
future1 = m1.make_future_dataframe(periods=1095)
forecast1 = m1.predict(future1)
daily_forecast_df = forecast1[['ds', 'yhat']].copy()
daily_forecast_df = daily_forecast_df.set_index('ds')
#daily_forecast_df.head(n=100)
common_kw = dict(x=daily_forecast_df.index, mode='lines')
data = [go.Scatter(y=daily_forecast_df[c], name=c, **common_kw) for c in daily_forecast_df.columns]
layout = dict(title="%s Prediction"%(key))
fig = dict(data=data, layout=layout)
iplot(fig, show_link=False)
    

/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


## Experimental Results

## Conclusions


## Bibliography
* https://en.wikipedia.org/wiki/Makridakis_Competitions
* https://eng.uber.com/forecasting-introduction/